In [1]:
import sys
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpim
from IPython.display import Image , HTML
import tldextract

In [29]:
def createDf(book_container):
    images_data = book_container.find_all('img')
    my_img_list = []
    for im in images_data:
        my_img_list.append(im)
    base_url = "http://books.toscrape.com/"
    ext_url = []    
    for i in range(len(my_img_list)):
        ext_url.append(my_img_list[i].attrs['src'])
    full_url = []
    for i in range(len(ext_url)):
        full_url.append(base_url+ ext_url[i])
    broken_images = []
    for img in full_url:
        file_name = img.split('/')[-1]
        # print(f"This is the file name: {file_name}")
        r1 = requests.get(img , stream=True)
        if r1.status_code == 200:
            with open(file_name,'wb') as f:
                for chunk in r1:
                    f.write(chunk)
        else:
            broken_images.append(img)
    # img = mpim.imread('2cdad67c44b002e7ead0cc35693c0e8b.jpg')
    # implot = plt.imshow(img)
    # plt.show()
    file_name = []
    for img in full_url:
        file_name.append(img.split('/')[-1])
        
    dataf = [[my_img_list[a].attrs['alt'] , '<img src = "{}"/>'.format(file_name[b]) ]for [a,b] in zip(range(len(my_img_list)),range(len(file_name)))]
    df = pd.DataFrame(dataf)
    df.columns = ['Title' , 'Cover']
    return HTML(df.to_html(escape= False))

In [30]:
test_page = requests.get('http://books.toscrape.com/')
soup_page = BeautifulSoup(test_page.content , 'html.parser')
spec = soup_page.find('div', class_="alert alert-warning")
book_container = spec.nextSibling.nextSibling
createDf(book_container=book_container)

,Title,Cover
0,A Light in the Attic,
1,Tipping the Velvet,
2,Soumission,
3,Sharp Objects,
4,Sapiens: A Brief History of Humankind,
5,The Requiem Red,
6,The Dirty Little Secrets of Getting Your Dream Job,
7,"The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull",
8,The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics,
9,The Black Maria,
